<a href="https://colab.research.google.com/github/Hassan-293/Tensorflow-Practice-Questions/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("/content/diamonds.csv")

In [4]:
df.shape

(53940, 11)

In [7]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0  carat        cut color clarity  depth  table  price     x  \
0               1   0.23      Ideal     E     SI2   61.5   55.0    326  3.95   
1               2   0.21    Premium     E     SI1   59.8   61.0    326  3.89   
2               3   0.23       Good     E     VS1   56.9   65.0    327  4.05   
3               4   0.29    Premium     I     VS2   62.4   58.0    334  4.20   
4               5   0.31       Good     J     SI2   63.3   58.0    335  4.34   
...           ...    ...        ...   ...     ...    ...    ...    ...   ...   
53935       53936   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75   
53936       53937   0.72       Good     D     SI1   63.1   55.0   2757  5.69   
53937       53938   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66   
53938       53939   0.86    Premium     H     SI2   61.0   58.0   2757  6.15   
53939       53940   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83   

     

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [9]:
df.isnull().sum()


Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = df.drop('price', axis=1), df[['price']]

In [13]:
X.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


In [14]:
y.head()

,price
0,326
1,326
2,327
3,334
4,335


In [17]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [19]:
cats

['cut', 'color', 'clarity']

In [18]:

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')
   print(X[col])

0            Ideal
1          Premium
2             Good
3          Premium
4             Good
           ...    
53935        Ideal
53936         Good
53937    Very Good
53938      Premium
53939        Ideal
Name: cut, Length: 53940, dtype: category
Categories (5, object): ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']
0        E
1        E
2        E
3        I
4        J
        ..
53935    D
53936    D
53937    D
53938    H
53939    D
Name: color, Length: 53940, dtype: category
Categories (7, object): ['D', 'E', 'F', 'G', 'H', 'I', 'J']
0        SI2
1        SI1
2        VS1
3        VS2
4        SI2
        ... 
53935    SI1
53936    SI1
53937    SI1
53938    SI2
53939    SI2
Name: clarity, Length: 53940, dtype: category
Categories (8, object): ['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2']


In [20]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [22]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [23]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [24]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

In [26]:
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 42.654


In [27]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

In [28]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

In [29]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:3938.68155	validation-rmse:3886.81385
[1]	train-rmse:2760.82164	validation-rmse:2724.20934
[2]	train-rmse:1935.57562	validation-rmse:1908.67677
[3]	train-rmse:1357.46154	validation-rmse:1338.40694
[4]	train-rmse:952.63039	validation-rmse:938.89031
[5]	train-rmse:669.37245	validation-rmse:659.13772
[6]	train-rmse:471.45067	validation-rmse:463.66706
[7]	train-rmse:333.34872	validation-rmse:327.50252
[8]	train-rmse:237.50392	validation-rmse:233.00799
[9]	train-rmse:171.66783	validation-rmse:168.04099
[10]	train-rmse:127.08221	validation-rmse:124.34036
[11]	train-rmse:97.60587	validation-rmse:95.36923
[12]	train-rmse:78.69947	validation-rmse:76.71299
[13]	train-rmse:66.93898	validation-rmse:65.36840
[14]	train-rmse:60.05171	validation-rmse:58.95693
[15]	train-rmse:56.08468	validation-rmse:55.20986
[16]	train-rmse:53.46550	validation-rmse:52.82029
[17]	train-rmse:51.43385	validation-rmse:51.14669
[18]	train-rmse:50.46262	validation-rmse:50.36928
[19]	train-rmse:49.81511	valid